In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
# config
embeddingSize=512
numPoints=200 # number of points that we are going to receive to make a prediction about f given x and y
numVars=2 # the dimenstion of input points x
numYs=1 # the dimension of output points y = f(x)
blockSize = 100 # spatial extent of the model for its context
batchSize = 128
dataInfo = 'XYE_{}Var_{}Points'.format(numVars, numPoints)
target = 'Skeleton' #'EQ'
dataFolder = '2Var_Rand_FixedLength_0to3_3.1to6'

In [5]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size, chars, target='EQ'):
        data_size, vocab_size = len(data), len(chars)
        print('data has %d examples, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        
        # padding token
        self.paddingToken = '_'
        self.paddingID = self.stoi[self.paddingToken]
        self.stoi[self.paddingToken] = self.paddingID
        self.itos[self.paddingID] = self.paddingToken
        self.threshold = [-1000,1000]
        
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data # it should be a list of examples
        self.target = target
    
    def __len__(self):
        return len(self.data)-1

    def __getitem__(self, idx):
        # grab an example from the data
        chunk = self.data[idx] # sequence of tokens including x, y, eq, etc.
        
        try:
            chunk = json.loads(chunk) # convert the sequence tokens to a dictionary
        except:
            print("Couldn't convert to json: {}".format(chunk))
            
        # encode every character in the equation to an integer
        # < is SOS, > is EOS
        dix = [self.stoi[s] for s in '<'+chunk[self.target]+'>']
        inputs = dix[:-1]
        outputs = dix[1:]
        
        # add the padding to the equations
        paddingSize = max(self.block_size-len(inputs),0)
        paddingList = [self.paddingID]*paddingSize
        inputs += paddingList
        outputs += paddingList 
        
        # make sure it is not more than what should be
        inputs = inputs[:self.block_size]
        outputs = outputs[:self.block_size]
        
        # extract points from the input sequence
        points = torch.zeros(numVars+numYs, numPoints)
        for idx, xy in enumerate(zip(chunk['X'], chunk['Y'])):
            x = xy[0] + [0]*(max(numVars-len(xy[0]),0)) # padding
            y = [xy[1]] if type(xy[1])== float else xy[1]
            y = y + [0]*(max(numYs-len(y),0)) # padding
            p = x+y # because it is only one point 
            p = torch.tensor(p)
            #replace nan and inf
            p = torch.nan_to_num(p, nan=0.0, 
                                 posinf=self.threshold[1], 
                                 neginf=self.threshold[0])
            p[p>self.threshold[1]] = self.threshold[1] # clip the upper bound
            p[p<self.threshold[0]] = self.threshold[0] # clip the lower bound
            points[:,idx] = p
        
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        inputs = torch.tensor(inputs, dtype=torch.long)
        outputs = torch.tensor(outputs, dtype=torch.long)
        return inputs, outputs, points


In [6]:
import json
from tqdm import tqdm
import glob
def processDataFiles(files):
    text = ''""
    for f in tqdm(files):
        with open(f, 'r') as h: 
            lines = h.read() # don't worry we won't run out of file handles
            if lines[-1]==-1:
                lines = lines[:-1]
            text += lines #json.loads(line)       
            
    return text

In [7]:
#path = 'D:\Datasets\Symbolic Dataset\Datasets\Mesh_Simple_GPT2_Sorted\TrainDatasetFixed\*.json'
path = 'D:/Datasets/Symbolic Dataset/Datasets/{}/Train/*.json'.format(dataFolder)
files = glob.glob(path)
text = processDataFiles(files)
chars = sorted(list(set(text))+['_','T','<','>']) # extract unique characters from the text before converting the text to a list
# T is for the test data
text = text.split('\n') # convert the raw text to a set of examples
text = text[:-1] if len(text[-1]) == 0 else text
train_dataset = CharDataset(text, blockSize, chars, target=target) 

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:01<00:00,  5.87it/s]


data has 110000 examples, 48 unique.


In [8]:
idx = np.random.randint(train_dataset.__len__())
inputs, outputs, points = train_dataset.__getitem__(idx)
print('inputs:{}'.format(inputs))
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

inputs:tensor([21, 43, 36, 39,  3, 12,  5, 23,  5, 45, 12,  6, 12,  5, 23,  4, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32])
id:15795
inputs:<sin(2*C*x2+2*C)____________________________________________________________________________________
outputs:sin(2*C*x2+2*C)>____________________________________________________________________________________
points:tensor([[ 1.3200,  0.8300,  1.4200,  1.6500,  2.8100,  0.0600,  2.3100,  2.1000,
          2.2800,  0.3000,  1.0200,  2.0800,  0.3800,  1.2700,  0.6700,  1.0000,
          1.8900,  1.4800,  1.7200,  1.6200,  0.0000,  1.0800,  1.2200,  0.6600,
          1.9600,  1.8200,  1.7500,  1.4000,  1.4500,  0.1500,  0.05

In [9]:
#path = 'D:\Datasets\Symbolic Dataset\Datasets\Mesh_Simple_GPT2_Sorted\TestDataset\*.json'
path = 'D:/Datasets/Symbolic Dataset/Datasets/{}/Test/*.json'.format(dataFolder)
files = glob.glob(path)
textTest = processDataFiles([files[0]])
textTest = textTest.split('\n') # convert the raw text to a set of examples
test_dataset = CharDataset(textTest, blockSize, chars)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 38.57it/s]

data has 1001 examples, 48 unique.


In [10]:
idx = np.random.randint(test_dataset.__len__())
inputs, outputs, points = test_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

tensor(0.0100) tensor(2.9800)
id:860
inputs:<sqrt(abs(0.41*x2-0.91))____________________________________________________________________________
outputs:sqrt(abs(0.41*x2-0.91))>____________________________________________________________________________
points:tensor([[2.5900, 2.4500, 2.0200, 2.0500, 1.6900, 2.6400, 0.2800, 1.7900, 1.3300,
         1.0000, 2.1700, 2.4700, 2.5800, 2.0600, 2.6100, 0.5400, 0.4200, 2.2500,
         2.4600, 0.2700, 1.1300, 0.1700, 1.4700, 0.4300, 0.3300, 0.0800, 0.4500,
         2.6000, 2.2800, 1.6200, 1.7300, 0.8400, 1.6200, 2.2900, 0.9600, 0.6400,
         1.7800, 1.1200, 0.6400, 0.7900, 2.8800, 1.9200, 1.8200, 0.6100, 0.3500,
         1.9000, 0.4100, 0.3700, 2.2000, 2.4700, 1.8000, 1.6200, 1.6900, 1.3200,
         0.5000, 1.3100, 2.3000, 0.6500, 2.9600, 2.9800, 2.7800, 0.7500, 2.1800,
         0.1700, 2.0000, 0.3400, 1.1600, 1.8500, 1.4000, 0.5500, 2.6200, 2.1500,
         1.3600, 2.6700, 2.1200, 1.5000, 1.5400, 2.1200, 0.8300, 0.2100, 0.5800,
         0

In [11]:
from mingpt.model import GPT, GPTConfig, PointNetConfig
pconf = PointNetConfig(embeddingSize=embeddingSize, 
                       numberofPoints=numPoints, 
                       numberofVars=numVars, 
                       numberofYs=numYs)
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=embeddingSize, padding_idx=train_dataset.paddingID)
model = GPT(mconf, pconf)

05/22/2021 02:33:52 - INFO - mingpt.model -   number of parameters: 3.058023e+07


In [ ]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=50, batch_size=batchSize, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*blockSize,
                      num_workers=0, ckpt_path='./SavedModels/bestModel/checkpoint.pt')
trainer = Trainer(model, train_dataset, test_dataset, tconf)

try:
    trainer.train()
except KeyboardInterrupt:
    print('KeyboardInterrupt')

  0%|                                                                                          | 0/860 [00:00<?, ?it/s]

In [13]:
# model.load_state_dict(torch.load('./SavedModels/bestModel/checkpoint.pt'))
# model = model.eval().to(trainer.device)

In [30]:
# add a safe wrapper for numpy math functions
from numpy import *
import numpy as np

def divide(x, y):
  x = np.nan_to_num(x)
  y = np.nan_to_num(y)
  return np.divide(x,y+1e-5)

def sqrt(x):
  x = np.nan_to_num(x)
  return np.sqrt(np.abs(x)) 

# Mean square error
def mse(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        our_sum += (y_hat[i] - y_gold[i]) ** 2

    return our_sum / len(y_gold)

# Mean square error
def relativeErr(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        if y_gold[i] < 1: 
            # use regular MSE
            our_sum += (y_hat[i] - y_gold[i]) ** 2
        else:
            # use relative MSE
            our_sum += ((y_hat[i] - y_gold[i])/y_gold[i]) ** 2

    return our_sum / len(y_gold)

In [ ]:
class pointsDataset(Dataset):

    def __init__(self, data):
        # data should be a list of x,y pairs
        self.x = data[0] # it should be a list
        self.y = data[1] # it should be a list
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # grab an example from the data
        x = self.x[idx] 
        y = self.y[idx] 
        
        x = torch.tensor(x, dtype=torch.float)
        y = torch.tensor(y, dtype=torch.float)
        return x,y

In [31]:
fName = '{}_SymbolicGPT_{}_{}_{}.txt'.format(dataInfo, 
                                             'GPT_PT_Summation', 
                                             'Padding',
                                             blockSize)

In [ ]:
# alright, let's sample some character-level symbolic GPT
from mingpt.utils import sample
from gp_model import Genetic_Model
from mlp_model import MLP_Model
    
loader = torch.utils.data.DataLoader(
                                test_dataset, 
                                shuffle=False, 
                                pin_memory=True,
                                batch_size=1,
                                num_workers=0)

testRange = [3.1,6.0]
numTestPoints = 10
#test = np.linspace(3.1,6.0,numTestPoints)

gpm = Genetic_Model(n_jobs=-1)
mlp = MLP_Model()

resultDict = {}
with open(fName, 'w', encoding="utf-8") as o:
    modelName = 'SymbolicGPT'
    resultDict[fName] = {modelName:[],
                         'GP':[],
                         'MLP':[]}
    
    for i, batch in enumerate(loader):
        inputs,outputs,points = batch
        
        print('Test Case {}.'.format(i))
        o.write('Test Case {}/{}.\n'.format(i,len(textTest)))
        
        t = json.loads(textTest[i])
        
        inputs = inputs[:,0:1].to(trainer.device)
        points = points.to(trainer.device)
        outputsHat = sample(model, inputs, blockSize, points=points,
                      temperature=1.0, sample=True, 
                      top_k=10)[0]
            
        # filter out predicted
        target = ''.join([train_dataset.itos[int(i)] for i in outputs[0]])
        predicted = ''.join([train_dataset.itos[int(i)] for i in outputsHat])
    
        target = target.strip(train_dataset.paddingToken).split('>')
        target = target[0] if len(target[0])>=1 else target[1]
        target = target.strip('<').strip(">")
        predicted = predicted.strip(train_dataset.paddingToken).split('>')
        predicted = predicted[0] if len(predicted[0])>=1 else predicted[1]
        predicted = predicted.strip('<').strip(">")
        
        # train a regressor to find the constants (too slow)
#         c = [0 for i,x in enumerate(predicted) if x=='C']
#         from scipy.optimize import least_squares
#         def lossFunc(constants, eq, X, Y):
#             err = 0
#             for x,y in zip(X,Y):
#                 eqTemp = eq.replace('C','{}').format(*constants)
#                 for i,e in enumerate(x):
#                     eqTemp = eqTemp.replace('x{}'.format(i+1), str(e))
#                 yHat = eval(eqTemp)
#                 err += (y-yHat)**2
#             err /= len(Y)
#             return err
        
#         try:
#             cHat = least_squares(lossFunc, c, args=(predicted, t['X'], t['Y']))
#             predicted = predicted.replace('C','{}').format(*cHat.x)
#         except:
#             print('Wrong Equation: {}'.format(predicted))
#             predicted = 0

        # let's enjoy GPU
        

       
        o.write('{}\n'.format(target))
        
        print('Target:{}\nOurs: {}'.format(target, predicted))
        
        Ys = [] #t['YT']
        Yhats = []
        for xs in t['XT']:
            try:
                eqTmp = target + '' # copy eq
                eqTmp = eqTmp.replace(' ','')
                eqTmp = eqTmp.replace('\n','')
                for i,x in enumerate(xs):
                    # replace xi with the value in the eq
                    eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                    if ',' in eqTmp:
                        assert 'There is a , in the equation!'
                YEval = eval(eqTmp)
                YEval = 0 if np.isnan(YEval) else YEval
                YEval = 100 if np.isinf(YEval) else YEval
            except:
                YEval = 100 #TODO: Maybe I have to punish the model for each wrong template not for each point
            Ys.append(YEval)
            try:
                eqTmp = predicted + '' # copy eq
                eqTmp = eqTmp.replace(' ','')
                eqTmp = eqTmp.replace('\n','')
                for i,x in enumerate(xs):
                    # replace xi with the value in the eq
                    eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                    if ',' in eqTmp:
                        assert 'There is a , in the equation!'
                Yhat = eval(eqTmp)
                Yhat = 0 if np.isnan(Yhat) else Yhat
                Yhat = 100 if np.isinf(Yhat) else Yhat
            except:
                Yhat = 100
            Yhats.append(Yhat)
        err = relativeErr(Ys,Yhats)
        
        if type(err) is np.complex128 or np.complex:
            err = abs(err.real)
            
        resultDict[fName][modelName].append(err)
        
        o.write('{}:{}\n{}\n\n'.format(modelName, 
                               err,
                               predicted))
        
        print('Err:{}'.format(err))
        
#         # tokenize to get input x, input y, and true eqn
#         train_data_x = t["X"]
#         train_data_y = t["Y"]
#         test_data_x = t["XT"]
#         test_data_y = t["YT"]

#         # train MLP model
#         mlp.reset()
#         model_eqn, _, best_err = mlp.repeat_train(
#                                                 train_data_x, 
#                                                 train_data_y,
#                                                 test_x=test_data_x, 
#                                                 test_y=test_data_y,                                     
#                                                 verbose=False)
#         test_y_hat = mlp.predict(test_data_x)
#         err = relativeErr(test_data_y,test_y_hat)
#         print("{}: {}".format(mlp.name, model_eqn)[:550])
#         print("Err: {:.5f}".format(err))
#         resultDict[fName]['MLP'].append(err)
#         o.write('\n{}: {}\n{}'.format('MLP', 
#                                    err,
#                                    model_eqn))

#         # train GPL model
#         gpm.reset()
#         model_eqn, _, best_err = gpm.repeat_train(train_data_x, train_data_y,
#                                                 test_x=test_data_x, test_y=test_data_y,
#                                                 verbose=False)
#         print("{}: {}".format(gpm.name, model_eqn)[:550])
#         test_y_hat = gpm.predict(test_data_x)
#         err = relativeErr(test_data_y,test_y_hat)
#         print("Err: {:.5f}".format(err))
#         resultDict[fName]['GP'].append(err)
#         o.write('\n{}: {}\n{}'.format('GP', 
#                                    err,
#                                    model_eqn))
        print('') # just an empty line
print('Avg Err:{}'.format(np.mean(resultDict[fName][modelName])))

Test Case 0.
Target:sin(x1+x2)
Ours: sqrt(abs(0.0*x1+0.0))*sin(0.0*x2+0.0)
Err:0.5202045708063546

Test Case 1.
Target:sqrt(abs(2*x1+0.28))
Ours: sqrt(abs(2.000411065654659*x1+0.2852733420245359))
Err:2.6644207189250187e-06

Test Case 2.
Target:sqrt(abs(x2-0.8))
Ours: sqrt(abs(0.4789842916039538*x2+0.14259501682528777))
Err:0.05245130616999702

Test Case 3.
Target:-0.19*x1*x2-0.29
Ours: sin(sin(0.21890616920167152*x2+-0.4664947976357132))
Err:1.0095608296425402

Test Case 4.
Target:x1+sqrt(abs(0.25*x2))
Ours: 1.2312631153171734*x2+0.22855858014690597
Err:0.0012156015340257227

Test Case 5.
Target:sqrt(abs(1.37*x2-0.64))
Ours: sqrt(abs(0.4798916482673865*x1+0.47989164762892905*x2+2*0.021574683203957556))
Err:0.036779191190833664

Test Case 6.
Target:-0.3*x1-0.45*x2-2.09
Ours: -0.1508181484028178*x1+2*-0.3016361023716106*x2+3*-0.6989643967457798
Err:0.00018240871952152502

Test Case 7.
Target:sqrt(abs(-0.64*x1*x2+0.92*x1-0.16*x2+0.22))
Ours: 0.19066557522157798*x1+0.19066557475981458*x2+

Target:0.65*x1**2
Ours: 2*0.5598838776005782**2*x1**2+4*0.10623306451347625**2*x1+2*0.01738761965254757**2
Err:0.0001821624282604149

Test Case 56.
Target:sqrt(abs(-0.14*x2-0.64))
Ours: sqrt(abs(sqrt(abs(0.23113090838283387*x2+0.38767084663922763))))
Err:1.9200364267967213e-05

Test Case 57.
Target:1.2*x1+1.94*x2
Ours: 1.691787883585299**2*x2**2+2*-1.2770949766235695**2*x2+-0.00018597464686062614**2+7.258049082607609*x1+-2.6331888169463444
Err:2.876397591388265

Test Case 58.
Target:sin(0.13*x1+0.34*x2+0.46)
Ours: sqrt(abs(sqrt(abs(0.4149163537498973*x2+-0.010068143760675944))))
Err:0.001955861333639438

Test Case 59.
Wrong Equation: C*x1+C*x2+2*C+sqrt(abs(C*x2+C))
Target:sqrt(abs(0.45-0.92*x1))
Ours: 0
Err:8144.797607937902

Test Case 60.
Target:sin(x1)
Ours: sin(sin(0.9765239613053492*x1+0.03283903208717799))
Err:0.008018243585963798

Test Case 61.
Target:sin(2*x2)
Ours: sin(-0.9997251452985204*x1+-0.9997251455228815*x2+2*1.5701841791760753)
Err:2.1071121059795791e-07

Test Case 62.


Target:-0.11*x1**2*sqrt(abs(0.46*x1))-0.04*x1
Ours: 0.0*x1*sin(0.0*x2+0.0)+0.0*sin(0.0*x2+0.0)
Err:0.29458069569443923

Test Case 112.
Wrong Equation: sqrt(abs(2*C**2*x1*x2+3*C**2*x2+2*C**2*x2+2*C**3**2)))
Target:0.31*x2**2
Ours: 0
Err:7457.995149667769

Test Case 113.
Target:sin(0.81*x1)
Ours: sqrt(abs(sin(0.8731108966861004*x2+-0.14623131121598162)))
Err:0.00907866901128533

Test Case 114.
Target:sqrt(abs(0.05*x2))
Ours: sqrt(abs(sin(0.04613399260433953*x2+-3.714082456375812e-11)))
Err:0.00012126462005881427

Test Case 115.
Target:sin(sin(x2))
Ours: -0.3826183216638578**2*x1*x2+1.1193579269949591e-08**2*x1+1.1193579269949591e-08**2*x2+-2.093220097445729e-09**2+sqrt(abs(0.8810603687715901*x1+-0.07020404316878162))
Err:0.00589680137161381

Test Case 116.
Target:sqrt(abs(0.95*x2**2))
Ours: 0.9716244714799532*x2+0.0018947672786205196
Err:3.2366569506349835e-06

Test Case 117.
Target:sqrt(abs(0.89*x2+0.44))
Ours: sqrt(abs(0.4432541240756626*x1+0.4432541240167441*x2+2*0.21811821039950405))

Target:-sin(0.65*x1**2+0.32*x1+0.03)
Ours: 0.0**2*x1**2*sin(0.0*x1+0.0)+2*0.0**2*x1*sin(0.0*x1+0.0)+0.0**2*sin(0.0*x1+0.0)
Err:0.4296472724440568

Test Case 169.
Target:0.99*x1+0.6
Ours: sqrt(abs(0.9160681937214908*x1+-0.12209572254744891))+sqrt(abs(0.916069114880673*x2+-0.12209584522400362))
Err:0.01434404838891793

Test Case 170.
Target:-sin(0.06*x2**2)
Ours: sin(-0.09372515716561944*x1+-0.09372515718970803*x2+2*0.04593931679316548)
Err:0.0016776219614127667

Test Case 171.
Target:1.52*x1+0.44
Ours: 1.199175047760888*x2+-0.37386263976507533+sqrt(abs(0.7947863852512345*x1+0.533638258028486))
Err:0.00040732884903972545

Test Case 172.
Target:x1*x2+0.28*x1+0.14*x2+sqrt(abs(x1+0.36))+0.04
Ours: 2*0.6523481397283589**2*x1*x2+2*0.5662210097291762**2*x1+2*0.5662210095585544**2*x2+2*0.499460680342113**2
Err:0.0021973619939973416

Test Case 173.
Target:sqrt(abs(2*x2+1.24))
Ours: sqrt(abs(1.9950176539823885*x2+1.2472226805799755))
Err:7.485808986302746e-07

Test Case 174.
Target:x2+sqrt(abs(0.

Target:sqrt(abs(sin(x2+0.47)))
Ours: sqrt(abs(sqrt(abs(-0.23861029402800663*x2+0.64398511243046))))
Err:0.022799653938157416

Test Case 223.
Target:x1+0.2*x2+sin(x1+0.67)
Ours: 0.9063832329834499*x2+0.2896149354998444+sin(0.9186483394687462*x2+0.401916896163425)
Err:0.0002472530955054088

Test Case 224.
Wrong Equation: C**4*x1**3*x2***2+C**4*x1+C*2*x1*x2+C*4*x1**3*x2**2+4*C**3*x1+2**C*4*x1*x2*x2*x1+2+3*C**4
Target:-0.56*x1*x2-0.64*x1+1.78*x2-0.24
Ours: 0
Err:10043.165603567191

Test Case 225.
Target:-sin(0.09*x2**2)
Ours: 0.0**2*x1*x2*sin(0.0*x1+0.0)+0.0**2*x1*sin(0.0*x1+0.0)+0.0**2*x2*sin(0.0*x1+0.0)+0.0**2*sin(0.0*x1+0.0)
Err:0.11687047524799386

Test Case 226.
Target:-0.4*x1*x2+0.18*x1+sqrt(abs(-0.28*x1-0.7))
Ours: -0.33954687780264786**2*x1**2+2*-1.483381644030463e-05**2*x1+1.2019583322677788e-06**2
Err:0.47725032225461894

Test Case 227.
Target:0.69*x2*sin(x2-0.11)-0.11
Ours: 2*0.0*x1*sin(0.0*x1+0.0)+2*0.0*sin(0.0*x1+0.0)
Err:0.5232592741197175

Test Case 228.
Target:sqrt(abs(0.51

Target:0.87*x2**2-0.82*x2+sqrt(abs(0.18*x2+0.93))
Ours: 0.7782621480263819**2*x1**2+2*-1.2727255963881001e-06**2*x1+1.9569886198139372e-08**2+0.14303464230539886*x2+0.44179847441885584
Err:0.0244259868532372

Test Case 277.
Target:sqrt(abs(x1-0.24*x2+0.55))
Ours: sqrt(abs(0.7619932170024524*x2+0.546966547425732))
Err:5.485628193904706e-07

Test Case 278.
Target:-0.85*x2**2+0.47*x2+0.28*sin(x1)
Ours: 0.0**2*x2**2*sin(0.0*x2+0.0)+2*0.0**2*x2*sin(0.0*x2+0.0)+0.0**2*sin(0.0*x2+0.0)
Err:6.428020575432603

Test Case 279.
Target:-0.46*x1*x2+1.5*x1+0.24*x2
Ours: sqrt(abs(1.3286888147134044*x1+-0.07972132415048308))+sin(-0.18167747629531364*x2+0.2096170831815523)
Err:0.03280324537495763

Test Case 280.
Target:x1+sin(0.76*x1)
Ours: sqrt(abs(1.1797558202727914**2*x1*x2+0.8173830570647261**2*x1+0.8173830582379868**2*x2+9.9505836582297e-05**2))
Err:0.007987536293323588

Test Case 281.
Target:0.58*x1**2*sin(x2+0.97)-0.8*x1
Ours: 0.0**2*x1**2*sqrt(abs(0.0*x2+0.0))+2*0.0**2*x1*sqrt(abs(0.0*x2+0.0))+0.

Target:1.86*x2**2+0.23*x2
Ours: 0.0**2*x2**2*sqrt(abs(0.0*x2+0.0))+2*0.0**2*x2*sqrt(abs(0.0*x2+0.0))+0.0**2*sqrt(abs(0.0*x2+0.0))
Err:0.8222299904934001

Test Case 332.
Target:-0.28*x2**2-0.36*x2+sqrt(abs(x1+0.17))
Ours: -0.38467132975286383**2*x1*x2+-8.555809594155266e-05**2*x1+-8.555726272134383e-05**2*x2+-1.295847493033675e-07**2
Err:0.19634306909090823

Test Case 333.
Target:x1*sqrt(abs(x1-0.81))+x2
Ours: 0.6872296920305958**2*x1*x2+0.3519649548436134**2*x1+0.6872296922113937**2*x2**2+3*0.173497475507171**2*x2+2*-8.675642396640199e-09**2
Err:0.04219950230604991

Test Case 334.
Target:sqrt(abs(x1))
Ours: sqrt(abs(0.9989946520270889*x2+-1.876558036047446e-09))
Err:2.1025800495612165e-07

Test Case 335.
Target:0.28*x1**2-0.56*x1+x2
Ours: sqrt(abs(1.0954238293288125**2*x1*x2+0.0002595267717001187**2*x1+0.0002595270741777913**2*x2+-0.38180989033159807**2))
Err:0.038919195363790425

Test Case 336.
Target:0.52*x2+sqrt(abs(0.98-0.66*x2))
Ours: sqrt(abs(2*0.6213158894348463*x1+2*0.198236912

Target:sqrt(abs(0.79*x1+x2-0.58))
Ours: sqrt(abs(1.412703732407858*x2+0.019748498855912632))
Err:0.028491612639228697

Test Case 384.
Target:sqrt(abs(x2**2-0.03*x2-0.01))
Ours: sqrt(abs(0.9890772872195713**2*x1*x2+0.029043433083612135**2*x1+0.029043433086629485**2*x2+0.03492107159536611**2))
Err:0.0003806939073710185

Test Case 385.
Target:x1-0.32*x2+0.67
Ours: sqrt(abs(2.018576776968502*x2+0.16197594084850694))
Err:0.005221880862234947

Test Case 386.
Target:x1*x2+0.55*x1
Ours: 0.7126253118744315**2*x1*x2+-1.1846236431182787e-09**2*x1+0.7126253120147626**2*x2**2+3*0.40393944154271616**2*x2+2*0.14178334804934967**2
Err:0.00014366701590554204

Test Case 387.
Target:1.18*x1
Ours: 2*0.5902058842447383*x1+2*0.0005273055657145899
Err:1.0041533208146965e-06

Test Case 388.
Target:0.76*x1*x2**2+0.7*x1
Ours: 0.0**3*x1**2*x2+0.0**3*x1**2+0.0**3*x1*x2**2+4*0.0**3*x1*x2+3*0.0**3*x1+0.0**3*x2**2+3*0.0**3*x2+2*0.0**3
Err:0.7881363539020378

Test Case 389.
Target:-0.07*x1*x2-0.1*x1
Ours: sin(0.0*x1+

Target:0.94*x2+sin(x1-0.47)+0.1
Ours: 0.7210862424221999*x2+-0.4514244432424078+sqrt(abs(1.6950375447433497*x2+-0.12773544157160233))
Err:0.0076266308839327305

Test Case 438.
Target:0.35*sin(0.53*x2)
Ours: sqrt(abs(0.04913450016654651*x1+-0.009227163939310746))
Err:0.0004852914562617558

Test Case 439.
Target:2*x2*sin(x2)-0.41
Ours: 0.0*x2*sin(0.0*x2+0.0)+0.0*sin(0.0*x2+0.0)
Err:0.7391073858388916

Test Case 440.
Target:sqrt(abs(2*x2))
Ours: sqrt(abs(2*0.9982013222598252*x1+2*-1.2956859468635384e-09))
Err:7.407418669249808e-07

Test Case 441.
Target:sqrt(abs(sqrt(abs(x1))))
Ours: sqrt(abs(sqrt(abs(0.9679927566378654*x2+-5.383850222297096e-10))))
Err:5.830983791103631e-05

Test Case 442.
Target:-1.33*x2-0.85
Ours: 2*-0.5466581420766387**2*x1**2+4*-0.0002251840119731205**2*x1+2*-6.50883289113371e-05**2
Err:1.423592043228268

Test Case 443.
Target:sin(sqrt(abs(0.76*x1-0.86)))
Ours: sqrt(abs(sqrt(abs(-0.24353400285418414*x2+0.27289209298252126))))
Err:0.005691938841078374

Test Case 444.


Target:sqrt(abs(0.11*x1*x2+0.06*x2))
Ours: sqrt(abs(0.15519639757330714*x1+0.15519639756839418*x2+2*-0.034650397012310125))
Err:0.0057776306970433455

Test Case 491.
Target:x2
Ours: sqrt(abs(0.9950709151518392**2*x1*x2+0.09872328819119564**2*x1+0.09872328841955055**2*x2+0.00013976611986307228**2))
Err:1.9314073760571424e-05

Test Case 492.
Target:1.36*x1**2+0.97*x2-0.14
Ours: 0.0**2*x2**2*sqrt(abs(0.0*x1+0.0))+2*0.0**2*x2*sqrt(abs(0.0*x1+0.0))+0.0**2*sqrt(abs(0.0*x1+0.0))
Err:0.8369603076678399

Test Case 493.
Target:x2*sqrt(abs(x2))
Ours: 1.7810828580720928*x2+-0.5905504614670405
Err:0.027109665874836784

Test Case 494.
Target:1.65*x1+0.14*x2+0.07
Ours: sqrt(abs(1.7832408550042826**2*x1*x2+0.3697997169315219**2*x1+0.3697997167832002**2*x2+0.04332543455491639**2))
Err:8.112125972924198e-06

Test Case 495.
Target:0.57*x1*x2+0.5*x1+sqrt(abs(0.41*x1))
Ours: 0.6284964825650506**2*x1**2+2*5.278787097768991e-09**2*x1+-2.9313274078779537e-08**2+1.3427444703341775*x2+-0.015757100431484407
Err:

In [ ]:
# train a mlp to find the constants

class MLP(nn.Module):
    def __init__(self, inputSize):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(inputSize, 100),
            nn.ReLU(),
            nn.Linear(100, inputSize)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
# plot the error frequency for model comparison
from matplotlib import pyplot as plt
num_eqns = len(resultDict[fName]['SymbolicGPT'])
num_vars = pconf.numberofVars

models = list(key for key in resultDict[fName].keys() if len(resultDict[fName][key])==num_eqns)
lists_of_error_scores = [resultDict[fName][key] for key in models if len(resultDict[fName][key])==num_eqns]
linestyles = ["-","dashdot","dotted","--"]

eps = 0.00001
y, x, _ = plt.hist([np.log([max(min(x+eps, 1e5),1e-5) for x in e]) for e in lists_of_error_scores],
                   label=models,
                   cumulative=True, 
                   histtype="step", 
                   bins=2000, 
                   density=True,
                   log=False)
y = np.expand_dims(y,0)
plt.figure(figsize=(15, 10))

for idx, m in enumerate(models): 
    plt.plot(x[:-1], 
           y[idx] * 100, 
           linestyle=linestyles[idx], 
           label=m)

plt.legend(loc="upper left")
plt.title("{} equations of {} variables".format(num_eqns, num_vars))
plt.xlabel("Log of Relative Mean Square Error")
plt.ylabel("Normalized Cumulative Frequency")

name = '{}.png'.format(fName)
plt.savefig(name)

In [ ]:
# benchmarks